In [1]:
import pandas as pd
import dwsemear

import postgresemear as sql  # mover os arquivos para a pasta principal

#dwsemear.atualize_metadados()

# GERANDO DADOS

In [13]:
dados = pd.read_csv('Acionamentos_01012019_a_30062019.csv', sep=';', encoding='latin-1', dtype=str)

In [14]:
dados = dwsemear.limpa_nome_das_colunas(dados)
dados = dwsemear.acionamentos_csv.corrige_campo_observacoes(dados)
dados.loc[:,'CPF CNPJ'] = dados.loc[:,'CPF CNPJ'].apply(dwsemear.cpf_para_inteiro)
dados = dwsemear.acionamentos_csv.geracao_coluna_datahora(dados)
dados = dwsemear.acionamentos_csv.adiciona_responsavel(dados)
dados.loc[:,'Dias atraso'] = dados.loc[:,'Dias atraso'].apply(dwsemear.converte_latin_para_float)

In [2]:
dwsemear.pegue_colunas('acionamentos')

['cpf',
 'datahora',
 'dias_atraso',
 'id',
 'motivo',
 'observacoes',
 'responsavel_id']

In [ ]:
dwsemear.pegue_colunas_naonulas('acionamentos') # o drop nas colunas nao nulas precisa ser consciente

# CARGA

In [32]:
id_acionamentos = sql.query_max_coluna('id', 'acionamentos')
if id_acionamentos == None:
    id_acionamentos = -1


registros = []
for i in dados.index:
    id_acionamentos += 1
    registros.append((
        dwsemear.para_int(dados.loc[i,'CPF CNPJ']),
        dwsemear.para_data(dados.loc[i,'datahora']),
        dwsemear.para_int(dados.loc[i,'Dias atraso']),
        id_acionamentos,
        dwsemear.para_str(dados.loc[i,'Motivo Contato']),
        dwsemear.para_str(dados.loc[i,'Observa es']),
        dwsemear.para_int(dados.loc[i,'responsavel_acionamento'])))

In [35]:
registros[0]

(62361643120,
 Timestamp('2019-02-19 12:46:00'),
 104,
 0,
 'CONTRA PROPOSTA',
 '19/02/2019 - Proposta negada. Contra proposta R$ 1.860,80. Cliente propos R$ 1.800,00',
 8)

In [31]:
# Resete a tabela
#dwsemear.delete_tabela('acionamentos')
#dwsemear.crie_tabela('acionamentos')

In [36]:
dwsemear.insira_tabela('acionamentos', registros)

In [3]:
dados_query = sql.query_para_pandas('''SELECT * FROM acionamentos, responsavel
                      WHERE acionamentos.responsavel_id = responsavel.id''')

In [4]:
dados_query.head(3)

,0,1,2,3,4,5,6,7,8,9,10
0,62361643120,2019-02-19 12:46:00,104,0,CONTRA PROPOSTA,19/02/2019 - Proposta negada. Contra proposta ...,8,Semear,Semear,8,Gabriel Torres de Carvalho Timochenco
1,97766330130,2019-02-19 12:53:00,315,1,CONTRA PROPOSTA,Proposta para quitaÃ§Ã£o da parcela 08 aceita ...,8,Semear,Semear,8,Gabriel Torres de Carvalho Timochenco
2,64756394604,2019-02-20 13:18:00,203,2,CONTRA PROPOSTA,Proposta para quitaÃ§Ã£o da parcela 06 aceita ...,8,Semear,Semear,8,Gabriel Torres de Carvalho Timochenco
